In [ ]:
import pandas as pd
import numpy as np

import snowflake.core
from snowflake.snowpark import Session
from snowflake.core import Root
import snowflake.snowpark as snowpark
from snowflake.snowpark.context import get_active_session
from snowflake.cortex import complete

from typing import List
import os
import sys
import json
import time
import requests

#Set up snowflake session vars and env vars
session = get_active_session()
root = Root(session)

In [ ]:
SELECT SCOPE:name, COUNT(*) FROM SNOWFLAKE.LOCAL.AI_OBSERVABILITY_EVENTS 
-- WHERE
-- ORDER BY TIMESTAMP DESC
GROUP BY SCOPE
-- limit 10;

In [ ]:
SELECT 
    TIMESTAMP AS TS,
    RECORD_ATTRIBUTES:"snow.ai.observability.object.name" AS AGENT_NAME,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.thread_id" AS THREAD_ID,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.parent_message_id" AS PARENT_MESSAGE_ID,
    VALUE:"snow.ai.observability.request_body"."messages"[0]."content"[0]."text" AS INPUT_QUERY,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.thinking_response" AS AGENT_PLANNING,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.cortex_analyst.sql_query" AS GENERATED_SQL,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.sql_execution.result" AS SQL_RESULT,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.cortex_search.results" AS CORTEX_SEARCH_RESULT,
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.tool.custom_tool.results" AS CUSTOM_TOOL_RESULT,
    RECORD_ATTRIBUTES:"ai.observability.record_root.output" AS AGENT_RESPONSE, 
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.model" AS REASONNG_MODEL, 
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.tool.name" AS AVAILABLE_TOOLS, 
    RECORD_ATTRIBUTES:"snow.ai.observability.agent.planning.tool_selection.name" AS TOOL_SELECTION,
    RECORD:"name" as OPERATION,
    
    FROM SNOWFLAKE.LOCAL.AI_OBSERVABILITY_EVENTS 
    WHERE SCOPE:name = 'snow.cortex.agent'
    AND AGENT_NAME = 'FINANCE_AGENT'
    AND OPERATION !='Agent'
    -- AND INPUT_QUERY IS NOT NULL
    AND THREAD_ID = '475131216265'
    ORDER BY THREAD_ID, TIMESTAMP, START_TIMESTAMP ASC;

In [ ]:
import json

df = cell2.to_pandas()

# df[df.OPERATION]
# df[df.OPERATION=='"CortexAnalystTool_SP500_Semantic_View_Tool"'].RECORD_ATTRIBUTES[19]
# json.loads(df.RECORD_ATTRIBUTES[20])["snow.ai.observability.agent.tool.cortex_search.results"]

In [ ]:
df.PARENT_MESSAGE_ID.bfill()
    # fillna(method='bfill')

In [ ]:
/* Generated by Snowflake Copilot */
SELECT
  observed_timestamp,
  start_timestamp,
  timestamp,
  record_type,
  value,
  CAST(
    GET_PATH (resource_attributes, 'service_name') AS TEXT
  ) AS service_name,
  CAST(
    GET_PATH (resource_attributes, 'service_version') AS TEXT
  ) AS service_version,
  CAST(
    GET_PATH (record_attributes, 'model_name') AS TEXT
  ) AS model_name,
  CAST(GET_PATH (record_attributes, 'prompt') AS TEXT) AS prompt,
  CAST(
    GET_PATH (record_attributes, 'completion') AS TEXT
  ) AS completion,
  CAST(GET_PATH (record, 'duration') AS FLOAT) AS duration,
  CAST(GET_PATH (record, 'status') AS TEXT) AS status
FROM
  SNOWFLAKE.LOCAL.ai_observability_events
WHERE
  timestamp >= DATEADD (DAY, -7, CURRENT_TIMESTAMP())
ORDER BY
  timestamp DESC;

In [ ]:
DB_NAME = "CHUNKING_EVAL_DEMO"
SCHEMA_NAME = "DATA"
STAGE_NAME = "DOCS"
WH_NAME = "CHUNKING_EVAL_WAREHOUSE"

In [ ]:
# Create the RAGWithObservability class to structure the RAG pipeline
from snowflake.cortex import complete


# ARCHIVE BELOW